# Análisis con XGBoost

¿MODIFICAR LOS ENFOQUES? y los títulos

En este notebook se encuentran dos enfoques de análisis: en un enfoque (por estudiante) los experimentos consideran que cada observación es la información de un estudiante en un cuatrimestre (el mismo estudiante puede aparecer en varios cuatrimestres); mientras que en el otro enfoque (por curso) las observaciones son los datos agrupados por curso.  

XGBoost puede encargarse internamente de las variables categóricas sin necesidad de hacer one-hot encoding. Debido a esto se realizaron análisis dejando que XGBoost trate a las variables categóricas como tales, pero también se probaron versiones con one-hot encoding de algunas variables para resaltar algunas características.

**Tabla de contenidos**<a id='toc0_'></a>    
- 1. [Importación de librerías y datos](#toc1_)    
- 2. [Abandona entre primer y segundo parcial](#toc2_)    
  - 2.1. [Modelo sin pa1](#toc2_1_)    
  - 2.2. [Modelo con pa1](#toc2_2_)    
  - 2.3. [Test de permutaciones](#toc2_3_)    
  - 2.4. [Repeticiones con diferentes semillas](#toc2_4_)    
    - 2.4.1. [Incluye SEDE, pa1, abandona1_p](#toc2_4_1_)    
    - 2.4.2. [Incluye SEDE y abandona1_p (sin pa1)](#toc2_4_2_)    
    - 2.4.3. [Incluye SEDE y pa1 (sin abandona1_p)](#toc2_4_3_)    
    - 2.4.4. [Incluye SEDE, sin pa1 y sin abandona1_p](#toc2_4_4_)    
    - 2.4.5. [sala, pa1 y abandona1_p](#toc2_4_5_)    
    - 2.4.6. [pa1 sin otras variables importantes](#toc2_4_6_)    
    - 2.4.7. [Sin pa1, sin pa1_prom y sin abandona1_p](#toc2_4_7_)    
    - 2.4.8. [Resumen comparación de variables importantes](#toc2_4_8_)    
  - 2.5. [Codificación one-hot](#toc2_5_)    
    - 2.5.1. [SEDE (y no sala)](#toc2_5_1_)    
    - 2.5.2. [sala (y no SEDE)](#toc2_5_2_)    
- 3. [Estudiantes con notas altas](#toc3_)    
  - 3.1. [Modelo con SEDE y pa1](#toc3_1_)    
  - 3.2. [Modelo con sala y pa1](#toc3_2_)    
  - 3.3. [Codificación one-hot: SEDE](#toc3_3_)    
  - 3.4. [Modelo sin pa1](#toc3_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Importación de librerías y datos](#toc0_)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import xgboost as xgb
import importlib

# Funciones para entrenar y evaluar los modelos.
import lib_entrenar as le
 
import matplotlib.pyplot as plt

# Descomentar lo siguiente para generar gráficos pgf:

# %matplotlib inline
# import matplotlib
# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

In [3]:
# Necesario cuando se realizan modificaciones en el módulo.
# importlib.reload(le)

In [6]:
df = pd.read_csv("../datos/dataset_02-feateng.csv")
df['SEDE'] = df['SEDE'].astype('str')
df['MATERIA'] = df['MATERIA'].astype('str')
df['edad'] = df['edad'].astype('category')
print("Cantidad de observaciones: ",len(df),"\n")
print("Variables: ",df.columns.values)
df['objetivo'] = (df['condición'] == 'Abandona2').astype(int)
df2 = df.loc[(df['valido2']==1) & (df['condición'] != 'Abandona1')].copy()

print("\nCantidad de observaciones válidas: ",len(df2))
print("Cantidad de cursos:", len(df2['curso'].unique()))
print("Positivos/Total: ", round(df2.objetivo.sum()/len(df2),3))

Cantidad de observaciones:  233615 

Variables:  ['anio' 'cuat' 'SEDE' 'MATERIA' 'pa1' 'pa2' 'Final' 'codCarrera'
 'facultad' 'rem1' 'rem2' 'estudiante' 'extranjero' 'curso' 'turno'
 'n_alum' 'p_ext' 'recurso' 'p_recursa' 'sala' 'pa1_prom' 'pa2_prom'
 'final_prom' 'edad' 'prom_edad' 'condición' 'abandona1_p' 'abandona2_p'
 'valido1' 'valido2']

Cantidad de observaciones válidas:  106987
Cantidad de cursos: 1763
Positivos/Total:  0.24


1763

## 2. <a id='toc2_'></a>[Abandona entre primer y segundo parcial](#toc0_)


Se seleccionan las observaciones válidas para este análisis, y todas las variables consistentes.

In [5]:
df['objetivo'] = (df['condición'] == 'Abandona2').astype(int)
df2 = df.loc[(df['valido2']==1) & (df['condición'] != 'Abandona1')].copy()

print("Cantidad de observaciones válidas: ",len(df2))
print("Positivos/Total: ", round(df2.objetivo.sum()/len(df2),3))

Cantidad de observaciones válidas:  106987
Positivos/Total:  0.24


Separar un 30% para conjunto de test y guardar los índices para utilizar los mismos conjuntos en otros scripts.  

In [6]:
# No ejecutar esta celda.
# Este código fue utilizado para generar los índices de los
# conjuntos de entrenamiento y test para algunos experimentos. 

# columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'codCarrera', 'facultad',
#         'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
#         'recurso', 'p_recursa', 'sala', 'abandona1_p']
# X = df2[columnas].copy()
# y = df2[['objetivo']].copy()
# print("Variables: ", X.columns.values)

# # Separar los datos.
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#         test_size=0.3,
#         random_state=1,
#         stratify=y)

# # Guardar los índices.
# selected = X_train.index.to_numpy()
# np.savetxt("indices_estudiante_train.txt.gz", selected)
# selected = X_test.index.to_numpy()
# np.savetxt("indices_estudiante_test.txt.gz", selected)

### 2.1. <a id='toc2_1_'></a>[Modelo sin pa1](#toc0_)

``sala`` pertenece a cada ``SEDE``, por lo cual ``SEDE`` puede ser considerado como una recategorización de ``sala``. Debido a esto se realizan dos pruebas con una y otra variable.  

Con la variable ``SEDE`` y **no ``sala``**, el modelo obtenido es el resultado del script <code>01a_estudiante.py</code>.

In [7]:
bst = xgb.Booster(model_file="modelo_01a_estudiante_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1_prom,28.353522
abandona1_p,14.268294
codCarrera,13.785037
prom_edad,9.372746
n_alum,9.170595
p_ext,8.891169
p_recursa,8.690249
edad,4.469163
recurso,3.479926
turno,2.755804


In [8]:
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1_prom', 'codCarrera',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'codCarrera', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  32097
Positivos/Total:  0.24
Average precision score: 0.359


El resultado es mejor que el de un clasificador aleatorio, aunque sea una mejora marginal.

Con la variable ``sala`` y **no ``SEDE``**, el modelo obtenido es el resultado del script <code>01b_estudiante.py</code>.

In [9]:
bst = xgb.Booster(model_file="modelo_01b_estudiante_sala_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1_prom,23.376151
codCarrera,9.849281
abandona1_p,9.689313
prom_edad,3.927636
edad,3.532655
p_recursa,3.525932
n_alum,3.487237
p_ext,3.477408
sala,2.494884
extranjero,2.153240


In [10]:
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'MATERIA', 'pa1_prom', 'codCarrera',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'sala', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'MATERIA', 'codCarrera', 'extranjero', 'edad',
       'turno', 'sala']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  32097
Positivos/Total:  0.24
Average precision score: 0.353


Incluyendo sala en lugar de SEDE se obtiene el mismo APS.

### 2.2. <a id='toc2_2_'></a>[Modelo con pa1](#toc0_)

Con la variable ``SEDE`` y **no ``sala``**, el modelo obtenido es el resultado del script <code>01d_estudiante.py</code>.

In [11]:
bst = xgb.Booster(model_file="modelo_01d_estudiante_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1,184.729453
abandona1_p,9.155655
prom_edad,4.914076
pa1_prom,4.366060
p_recursa,4.258887
p_ext,3.707278
codCarrera,3.249850
edad,3.016781
n_alum,2.794579
turno,2.658136


Según XGBoost, la importancia de la variable ``pa1`` es dominante.

In [12]:
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'codCarrera',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'codCarrera', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  32097
Positivos/Total:  0.24


Average precision score: 0.594


### 2.3. <a id='toc2_3_'></a>[Test de permutaciones](#toc0_)

Se analizan las importancias mediante el test de permutaciones.  

Línea de base: modelo sin ``pa1``.

In [19]:
bst = xgb.Booster(model_file="modelo_01a_estudiante_aucpr.json")
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1_prom', 'codCarrera',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'codCarrera', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)
y_pred = bst.predict(dtest)
base = average_precision_score(y_test, y_pred)

importancias = np.array([])
for col in columnas:
    Xsave = X_test[col].copy()
    X_test[col] = np.random.permutation(X_test[col])
    for col in cats:
        X_test[col] = X_test[col].astype('category')
    dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)
    y_pred = bst.predict(dtest)
    aps = average_precision_score(y_test, y_pred)
    importancias = np.concatenate([importancias, [round(base - aps,2)]])
    X_test[col] = Xsave
df_importancias = pd.DataFrame({'variable':columnas, 'importancia':importancias})
df_importancias.sort_values(by='importancia',ascending=False).reset_index(drop=True)

,variable,importancia
0,p_recursa,0.12
1,abandona1_p,0.12
2,edad,0.11
3,prom_edad,0.11
4,turno,0.11
5,n_alum,0.11
6,p_ext,0.11
7,recurso,0.11
8,extranjero,0.09
9,codCarrera,0.08


Línea de base: modelo con ``pa1``.

In [20]:
bst = xgb.Booster(model_file="modelo_01d_estudiante_aucpr.json")
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'codCarrera',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'codCarrera', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)
y_pred = bst.predict(dtest)
base = average_precision_score(y_test, y_pred)

importancias = np.array([])
for col in columnas:
    Xsave = X_test[col].copy()
    X_test[col] = np.random.permutation(X_test[col])
    for col in cats:
        X_test[col] = X_test[col].astype('category')
    dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)
    y_pred = bst.predict(dtest)
    aps = average_precision_score(y_test, y_pred)
    importancias = np.concatenate([importancias, [round(base - aps,2)]])
    X_test[col] = Xsave
df_importancias = pd.DataFrame({'variable':columnas, 'importancia':importancias})
df_importancias.sort_values(by='importancia',ascending=False).reset_index(drop=True)

,variable,importancia
0,pa1_prom,0.35
1,edad,0.35
2,prom_edad,0.35
3,turno,0.35
4,n_alum,0.35
5,p_ext,0.35
6,recurso,0.35
7,p_recursa,0.35
8,abandona1_p,0.35
9,codCarrera,0.34


### 2.4. <a id='toc2_4_'></a>[Repeticiones con diferentes semillas](#toc0_)

**Es necesario ejecutar nuevamente estos entrenamientos porque se cambió la forma de medir importancias con XGBoost.**

En cada experimento se repite el entrenamiento 10 veces, con diferentes conjuntos de entrenamiento y prueba, para generar estadística y al mismo tiempo verificar si el orden de importancia de las variables cambia según los ejemplos que se utilicen para el entrenamiento.  

En cada experimento también se seleccionan distintas variables.

#### 2.4.1. <a id='toc2_4_1_'></a>[Incluye SEDE, pa1, abandona1_p](#toc0_)

Los resultados se obtuvieron con el script <code>02a_repeticiones.py</code>


In [11]:
aps = np.loadtxt("salida_02a_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.591 +/- 0.006


APS bastante alejado del clasificador aleatorio.  

Importancias relativas de las variables en cada experimento:

In [12]:
importancias = pd.read_csv("salida_02a_importancias.csv")
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias[str(i)].min(),1)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,9.2,10.0,9.8,8.9,11.5,8.3,16.1,11.9,10.3,10.9
1,pa1_prom,8.7,9.6,8.9,7.0,9.6,4.5,16.3,11.8,10.1,9.2
2,prom_edad,7.8,9.5,8.0,6.2,8.9,6.0,14.7,11.4,8.7,7.5
3,pa1,6.4,5.8,5.6,4.0,5.2,8.2,9.3,4.1,4.4,7.8
4,p_ext,6.2,8.5,6.0,5.9,9.3,3.0,12.1,10.4,7.8,7.0
5,n_alum,5.8,7.8,6.0,5.7,7.4,3.5,11.1,10.2,7.3,6.1
6,codCarrera,5.6,6.8,3.9,3.5,4.9,3.3,6.4,7.2,4.6,4.6
7,p_recursa,5.6,9.0,6.6,5.7,8.1,3.0,11.4,9.6,7.5,7.1
8,facultad,3.4,4.0,1.9,2.5,3.0,1.3,5.1,4.4,2.4,2.9
9,edad,2.8,3.0,2.2,2.3,2.8,2.7,4.1,2.8,2.3,3.4


Orden de importancias:

In [13]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
importancias_rank.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,16.0,16.0,16.0,16.0,16.0,16.0,15.0,16.0,16.0,16.0
1,pa1_prom,15.0,15.0,15.0,15.0,15.0,13.0,16.0,15.0,15.0,15.0
2,prom_edad,14.0,14.0,14.0,14.0,13.0,14.0,14.0,14.0,14.0,13.0
3,pa1,13.0,9.0,10.0,10.0,10.0,15.0,10.0,8.0,9.0,14.0
4,p_ext,12.0,12.0,11.5,13.0,14.0,9.5,13.0,13.0,13.0,11.0
5,n_alum,11.0,11.0,11.5,11.5,11.0,12.0,11.0,12.0,11.0,10.0
6,codCarrera,9.5,10.0,9.0,9.0,9.0,11.0,9.0,10.0,10.0,9.0
7,p_recursa,9.5,13.0,13.0,11.5,12.0,9.5,12.0,11.0,12.0,12.0
8,facultad,8.0,8.0,5.0,8.0,7.0,6.0,8.0,9.0,8.0,7.0
9,edad,7.0,7.0,6.5,7.0,6.0,7.5,7.0,5.0,7.0,8.0


In [14]:
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,1,1,1,1,1,1,2,1,1,1
1,n_alum,,,5,5,,5,,5,,
2,p_ext,5,5,,4,3,,4,4,4,
3,p_recursa,,4,4,,5,,5,,5,5
4,pa1,4,,,,,2,,,,3
5,pa1_prom,2,2,2,2,2,4,1,2,2,2
6,prom_edad,3,3,3,3,4,3,3,3,3,4


La tabla muestra las 5 variables más importantes en cada modelo de las 10 repeticiones. (1 es el más importante).  
En todas las repeticiones se tiene a abandona1_p, pa1_prom y prom_edad entre las 4 más importantes.

#### 2.4.2. <a id='toc2_4_2_'></a>[Incluye SEDE y abandona1_p (sin pa1)](#toc0_)

Los resultados se obtuvieron con el script <code>02b_repeticiones.py</code>

In [15]:
aps = np.loadtxt("salida_02b_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.374 +/- 0.005


In [16]:
importancias = pd.read_csv("salida_02b_importancias.csv")
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias[str(i)].min(),1)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,9.2,9.2,13.3,9.8,9.2,17.0,11.3,9.9,8.2,13.1
1,pa1_prom,8.4,8.1,11.6,9.4,8.5,17.1,9.8,9.9,8.0,9.9
2,codCarrera,7.3,7.9,12.3,8.3,8.8,18.4,9.3,8.3,5.6,10.9
3,prom_edad,6.8,7.1,8.5,7.8,6.8,10.4,9.3,8.8,7.6,8.8
4,n_alum,5.6,5.6,6.2,6.5,5.4,8.7,6.7,7.4,5.8,7.0
5,p_ext,4.8,5.8,7.5,5.7,5.7,8.9,6.8,7.3,6.2,6.2
6,p_recursa,4.8,5.2,6.5,5.4,5.1,9.0,6.7,6.8,5.2,7.4
7,facultad,3.5,3.6,5.8,3.8,4.2,8.9,4.9,3.7,2.4,5.4
8,edad,3.2,3.0,5.4,4.0,4.4,9.1,4.0,3.9,2.6,5.1
9,recurso,2.4,2.6,3.9,2.7,2.6,5.3,2.8,3.5,2.3,3.3


In [17]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,1,1,1,1,1,3,1,2,1,1
1,codCarrera,3,3,2,3,2,1,3,4,,2
2,edad,,,,,,5,,,,
3,n_alum,5,,,5,,,,5,5,
4,p_ext,,5,5,,5,,5,,4,
5,p_recursa,,,,,,,,,,5
6,pa1_prom,2,2,3,2,3,2,2,1,2,3
7,prom_edad,4,4,4,4,4,4,4,3,3,4


#### 2.4.3. <a id='toc2_4_3_'></a>[Incluye SEDE y pa1 (sin abandona1_p)](#toc0_)

Los resultados se obtuvieron con el script <code>02c_repeticiones.py</code>

In [18]:
aps = np.loadtxt("salida_02c_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.586 +/- 0.005


In [19]:
importancias = pd.read_csv("salida_02c_importancias.csv")
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias[str(i)].min(),1)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,pa1_prom,16.8,12.8,12.0,11.2,12.0,8.6,9.0,9.9,9.8,12.5
1,p_ext,16.8,9.5,9.9,8.5,11.0,7.2,6.8,9.8,8.4,10.4
2,prom_edad,13.9,12.6,12.4,10.2,12.7,10.8,9.8,10.4,9.6,12.7
3,p_recursa,13.1,9.9,6.3,7.0,10.2,7.9,6.7,7.7,7.9,11.1
4,n_alum,12.8,9.2,9.0,8.2,10.4,7.7,7.2,7.3,7.7,10.0
5,pa1,10.2,4.2,10.3,13.2,4.6,7.9,5.0,3.9,4.3,6.3
6,codCarrera,8.0,5.1,5.1,6.0,8.3,5.0,4.0,4.6,3.9,6.1
7,facultad,5.6,3.0,3.1,4.2,4.7,2.0,2.0,2.4,2.2,4.0
8,edad,4.5,2.4,4.1,3.8,3.0,3.7,2.2,2.3,2.4,3.3
9,SEDE,4.3,2.8,2.3,1.8,3.9,2.2,2.6,2.4,2.3,3.8


In [20]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,n_alum,5,5,5,5,4,5,3,5,5,5
1,p_ext,2,4,4,4,3,,4,3,3,4
2,p_recursa,4,3,,,5,4,5,4,4,3
3,pa1,,,3,1,,3,,,,
4,pa1_prom,1,1,2,2,2,2,2,2,1,2
5,prom_edad,3,2,1,3,1,1,1,1,2,1


#### 2.4.4. <a id='toc2_4_4_'></a>[Incluye SEDE, sin pa1 y sin abandona1_p](#toc0_)

Los resultados se obtuvieron con el script <code>02d_repeticiones.py</code>

In [21]:
aps = np.loadtxt("salida_02d_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.37 +/- 0.004


In [22]:
importancias = pd.read_csv("salida_02d_importancias.csv")
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias[str(i)].min(),1)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,prom_edad,13.8,11.4,9.6,12.3,15.2,11.7,13.2,8.8,11.6,12.2
1,pa1_prom,12.9,11.0,25.4,11.4,24.0,10.0,13.4,8.9,10.7,10.8
2,codCarrera,11.3,7.5,23.0,8.0,17.8,9.0,10.4,6.3,7.6,7.9
3,p_ext,10.6,9.7,10.8,10.1,13.8,8.3,12.2,8.2,9.9,10.2
4,n_alum,10.0,9.1,10.0,10.3,10.2,8.5,11.2,8.0,9.1,9.7
5,p_recursa,9.4,9.2,8.8,9.1,11.2,8.1,9.8,8.0,9.2,8.9
6,facultad,5.1,3.6,11.0,3.8,9.8,4.4,5.6,3.4,3.7,3.9
7,edad,4.0,3.6,11.8,2.9,11.5,4.0,4.3,3.1,2.9,3.4
8,SEDE,3.6,3.4,2.8,3.5,3.5,3.5,4.9,2.4,3.3,4.0
9,recurso,3.5,2.9,4.6,3.1,5.0,2.7,3.5,2.3,3.0,3.3


In [23]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,codCarrera,3,,2,,2,3,5,,,
1,edad,,,3,,5,,,,,
2,facultad,,,4,,,,,,,
3,n_alum,5,5,,3,,4,4,4,5,4
4,p_ext,4,3,5,4,4,5,3,3,3,3
5,p_recursa,,4,,5,,,,5,4,5
6,pa1_prom,2,2,1,2,1,2,1,1,2,2
7,prom_edad,1,1,,1,3,1,2,2,1,1


#### 2.4.5. <a id='toc2_4_5_'></a>[sala, pa1 y abandona1_p](#toc0_)

Resultados del script <code>02e_repeticiones.py</code>.

In [24]:
aps = np.loadtxt("salida_02e_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.591 +/- 0.004


In [25]:
importancias = pd.read_csv("salida_02e_importancias.csv")
importancias = importancias.fillna(0)
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias.loc[importancias[str(i)] > 0, str(i)].min(),2)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,9.72,8.87,9.85,10.05,7.93,10.71,10.68,13.90,10.44,19.4
1,pa1_prom,7.17,7.70,8.17,7.93,7.14,8.08,7.50,7.14,8.89,16.0
2,p_ext,6.72,6.09,5.37,5.88,5.29,5.75,4.55,7.10,5.89,12.8
3,prom_edad,6.67,6.83,7.72,7.95,6.86,8.08,6.87,8.80,8.00,13.6
4,p_recursa,6.42,6.00,5.78,6.11,5.14,6.17,5.45,7.31,6.00,7.8
5,n_alum,6.08,5.70,5.93,5.84,4.79,6.08,5.13,6.22,5.67,10.2
6,pa1,5.56,5.35,4.84,5.16,5.00,6.38,9.03,9.45,8.33,12.8
7,codCarrera,5.25,4.48,3.58,3.45,3.14,4.46,3.90,5.35,3.78,5.4
8,sala,4.31,4.22,4.55,4.75,3.64,6.08,4.27,4.43,4.22,9.0
9,edad,3.31,2.26,2.17,2.77,2.00,3.08,4.32,3.45,3.44,4.6


In [26]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,1,1,1,1,1,1,1,1,1,1
1,n_alum,,,4,,,,,,,
2,p_ext,3,4,,5,4,,,,,5
3,p_recursa,5,5,5,4,5,5,5,4,5,
4,pa1,,,,,,4,2,2,3,4
5,pa1_prom,2,2,2,3,2,2,3,5,2,2
6,prom_edad,4,3,3,2,3,3,4,3,4,3


#### 2.4.7. <a id='toc2_4_7_'></a>[Sin pa1, sin pa1_prom y sin abandona1_p](#toc0_)

Resultados del script <code>02g_repeticiones.py</code>.

In [31]:
aps = np.loadtxt("salida_02g_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.358 +/- 0.004


In [32]:
importancias = pd.read_csv("salida_02g_importancias.csv")
importancias = importancias.fillna(0)
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias.loc[importancias[str(i)] > 0, str(i)].min(),2)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,p_ext,11.99,13.07,13.4,16.00,18.13,9.18,15.92,16.89,14.61,15.20
1,p_recursa,10.98,11.82,10.0,16.21,17.12,8.65,15.79,15.21,15.54,15.31
2,n_alum,10.71,11.18,7.9,14.37,16.57,8.82,15.32,15.28,12.61,12.37
3,sala,9.41,9.36,8.9,11.11,12.28,7.71,11.48,11.01,12.00,10.59
4,codCarrera,7.67,6.57,5.3,9.84,10.90,6.47,13.69,13.58,7.68,9.27
5,facultad,3.47,3.32,3.7,5.58,4.81,4.41,5.77,5.63,3.64,5.18
6,recurso,2.30,2.41,1.9,4.37,4.37,2.29,5.65,5.50,3.00,3.47
7,turno,2.04,2.07,1.4,3.42,3.57,1.88,4.07,3.78,2.89,3.27
8,edad,2.00,2.09,2.3,4.32,4.33,2.18,4.37,4.66,3.32,3.43
9,cuat,1.73,1.34,1.1,1.84,1.86,1.12,1.73,1.74,1.36,1.55


In [33]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,codCarrera,5,5,5,5,5,5,4,4,5,5
1,n_alum,3,3,4,3,3,2,3,2,3,3
2,p_ext,1,1,1,2,1,1,1,1,2,2
3,p_recursa,2,2,2,1,2,3,2,3,1,1
4,sala,4,4,3,4,4,4,5,5,4,4


### 2.5. <a id='toc2_5_'></a>[Codificación one-hot](#toc0_)

Codificación "one-hot" de las variables categóricas, para discriminar importancias.

Se realizan 5 repeticiones en cada experimento, y una búsqueda más acotada de hiperparámetros.

#### 2.5.1. <a id='toc2_5_1_'></a>[SEDE (y no sala)](#toc0_)

Discriminando las 6 sedes más grandes y otras (O).

Resultados obtenidos con el script <code>03a_dummies.py</code>

#### 2.5.2. <a id='toc2_5_2_'></a>[sala (y no SEDE)](#toc0_)

Resultados obtenidos con el script <code>03b_dummies.py</code>

In [35]:
aps = np.loadtxt("salida_03a_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.594 +/- 0.005


In [36]:
importancias = pd.read_csv("salida_03a_importancias.csv")
importancias = importancias.fillna(0)
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias.loc[importancias[str(i)] > 0, str(i)].min(),2)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5
0,abandona1_p,552.0,199.0,1494.0,474.0,656.0
1,pa1_prom,534.0,197.0,1222.0,407.0,564.0
2,prom_edad,501.0,159.0,1241.0,396.0,522.0
3,p_ext,475.0,180.0,970.0,342.0,487.0
4,p_recursa,455.0,167.0,984.0,392.0,449.0
...,...,...,...,...,...,...
72,facultad_CIENCIAS ECONOMICAS,1.0,4.0,7.0,1.0,4.0
73,codCarrera_19,1.0,8.0,14.0,3.0,13.0
74,codCarrera_20,1.0,1.0,8.0,2.0,0.0
75,codCarrera_65,1.0,5.0,6.0,1.0,10.0


In [37]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
df_top10 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:10]])
df_top10['variable'] = np.unique(top)
df_top10 = df_top10.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:10]
    df_top10[str(i+1)] = ''
    for j in range(len(top)):
        df_top10.loc[df_top10['variable'] == top[j],str(i+1)] = str(j+1)
df_top10

KeyError: '6'

Se repite el resultado sobre las 5 variables más importantes en cada modelo.

Las dummies no parecen agregar detalle a los resultados.

## 3. <a id='toc3_'></a>[Estudiantes con notas altas](#toc0_)

Se restringe el análisis a las observaciones con calificaciones en el parcial 1 mayores a 3.

In [38]:
df_altas = df.loc[df['pa1'] > 3].copy()
df_altas = df_altas.loc[(df_altas['valido2']==1) & (df_altas['condición'] != 'Abandona1')].copy()
df_altas['objetivo'] = df_altas['objetivo'].astype('int')
df_altas = df_altas.reset_index(drop=True)
print("\nCantidad de observaciones con notas > 3: ", len(df_altas))
print("Positivos/Total: ", round(df_altas.objetivo.sum()/len(df_altas),3))


Cantidad de observaciones con notas > 3:  51789
Positivos/Total:  0.042


Separar un 30% para conjunto de test y guardar los índices para utilizar los mismos conjuntos en otros scripts.

In [ ]:
# No ejecutar esta celda.
# Este código fue utilizado para generar los índices de los
# conjuntos de entrenamiento y test para algunos experimentos. 
# df_altas = df_altas[['cuat', 'SEDE', 'MATERIA', 'pa1', 'codCarrera',
#         'facultad', 'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso',
#         'p_recursa', 'sala', 'pa1_prom', 'edad', 'prom_edad',
#         'abandona1_p','objetivo']]
# df_altas.to_csv('../datos/dataset_30-notas_altas.csv', index=False)

# X = df_altas.drop(['objetivo'], axis=1).copy()
# y = df_altas[['objetivo']].copy()

# # Separar los datos.
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#         test_size=0.3,
#         random_state=1,
#         stratify=y)

# # Guardar los índices.
# selected = X_train.index.to_numpy()
# np.savetxt("indices_notas_altas_train.txt.gz", selected)
# selected = X_test.index.to_numpy()
# np.savetxt("indices_notas_altas_test.txt.gz", selected)

### 3.1. <a id='toc3_1_'></a>[Modelo con SEDE y pa1](#toc0_)

Con la variable ``SEDE`` y **no ``sala``**, el modelo obtenido es el resultado del script <code>30_altas.py</code>.

In [39]:
bst = xgb.Booster(model_file="modelo_30_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1,8.727299
abandona1_p,7.560513
pa1_prom,6.460779
prom_edad,6.214392
p_recursa,5.515126
p_ext,5.448417
n_alum,4.604799
SEDE,2.369530
MATERIA,2.038714
edad,1.907328


In [40]:
conj_test = np.loadtxt("indices_notas_altas_test.txt.gz")
df2_test = df_altas.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'facultad',
       'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso', 'p_recursa',
       'pa1_prom', 'edad', 'prom_edad', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  15537
Positivos/Total:  0.042
Average precision score: 0.126


Es un resultado marginal, pero está por encima del aleatorio.

### 3.2. <a id='toc3_2_'></a>[Modelo con sala y pa1](#toc0_)

Con la variable ``sala`` y **no ``SEDE``**, el modelo obtenido es el resultado del script <code>31_altas.py</code>.

In [41]:
bst = xgb.Booster(model_file="modelo_31_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1,11.685932
abandona1_p,10.413203
prom_edad,8.993553
pa1_prom,7.716251
n_alum,6.850746
p_recursa,6.548778
p_ext,5.793159
sala,5.188944
facultad,2.868785
turno,2.647928


In [42]:
conj_test = np.loadtxt("indices_notas_altas_test.txt.gz")
df2_test = df_altas.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'sala', 'MATERIA', 'pa1', 'facultad',
       'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso', 'p_recursa',
       'pa1_prom', 'edad', 'prom_edad', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'sala', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  15537
Positivos/Total:  0.042
Average precision score: 0.132


### 3.3. <a id='toc3_3_'></a>[Codificación one-hot: SEDE](#toc0_)

Codificación one-hot de las variables categóricas. Con SEDE y no sala. El modelo obtenido es el resultado del script <code>32_altas.py</code>.

In [43]:
bst = xgb.Booster(model_file="modelo_32_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
abandona1_p,1049.700209
pa1,1005.418571
prom_edad,933.778332
pa1_prom,829.358385
p_recursa,738.856722
n_alum,663.016564
p_ext,627.340724
edad_3,249.261086
recurso,220.080976
MATERIA,185.917601


### 3.4. <a id='toc3_4_'></a>[Modelo sin pa1](#toc0_)

Resultado del script <code>33_altas.py</code>.

In [44]:
bst = xgb.Booster(model_file="modelo_33_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
abandona1_p,11.165182
prom_edad,11.009246
pa1_prom,9.440964
p_recursa,8.909869
p_ext,7.324520
n_alum,6.565862
SEDE,4.466109
turno,3.266483
facultad,3.176499
edad,3.104831


In [45]:
conj_test = np.loadtxt("indices_notas_altas_test.txt.gz")
df2_test = df_altas.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'facultad',
       'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso', 'p_recursa',
       'pa1_prom', 'edad', 'prom_edad', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  15537
Positivos/Total:  0.042
Average precision score: 0.108
